## Demos of ErrorHandle and Preflight Checks
This notebook demonstrates three ways of messaging a script's user regarding error conditions. The ErrorHandle class can manage checking and reporting general conditions within a program. Preflight (preflight.py) is a special case that uses ErrorHandle and generic functions to precheck input files (preflight.CheckExcelFiles class) and input DataFrames (preflight.CheckDataFrame class). All use cases rely on the table in admin file ErrorCodes.xlsx to look up messages. </br></br>
JDL 2/12/24

In [1]:
import os, sys
import pandas as pd
import logging
logging.basicConfig(level=logging.ERROR, filename='demo.log', format='%(message)s')

path_libs = os.getcwd() + os.sep + 'libs' + os.sep
if not path_libs in sys.path: sys.path.append(path_libs)

#Add the libs subdirectory to sys.path and import the libraries
from libs.error_handling import ErrorHandle
from libs.preflight import CheckDataFrame
import util


### Case 1: Error Handling in procedural code
* Instance ErrorHandle class to use for checking conditions during processing
* Set .Locn for error message lookup by code. Base and code-speciifc rows are in ErrorCodes.xlsx
* This example illustrates using Python logging to write to a log file (demo.log initialized above in the `logging.basicConfig()` command). It reinitializes the file at the beginning of the cell


In [2]:
errs = ErrorHandle(path_libs, ErrMsgHeader='Procedural Demo', IsLog=True)
errs.reset_log_file(logging.getLogger())

#This string is the lookup key for getting the base error code from ErrorCodes.xlsx
errs.Locn = 'ProceduralDemo'

# Example procedural code with an error check
x = 2
y = 3
sum = x + y

#Check that x+ y is less than 4 --
#"1" is the routine-specific error code (added to the self.Locn base error code to get the lookup error code)
if errs.is_fail(sum > 4, 1, err_param=str(sum)): errs.RecordErr()
    #Insert other steps to close things down if fatal error was deetected

Procedural Demo
ERROR: x + y should be less than or equal to 4. Sum: 5


### Case 2: Error Handling within a Class
* Passing Locn argument as util.current_fn() tells .is_fail to look up the error by name of the function where the error occurred
* errs can either be instanced in __init__() or [better for toggling between test and production] instanced externally and passed to __init__() as an argument

In [3]:
class SumAndProduct():
    def __init__(self, path_libs, x, y):
        self.x = x
        self.y = y
        self.sum = None
        self.product = None
        self.errs = ErrorHandle(path_libs, ErrMsgHeader='', 
                                IsHandle=True, IsLog=True)
        self.errs.reset_log_file(logging.getLogger())

    @property
    def procedure_to_do_all_steps(self):
        self.calculate_sum()
        if not self.errs.IsErr: self.calculate_product()
        return self.sum, self.product

    def calculate_sum(self):
        self.sum = self.x + self.y
        if self.errs.is_fail(self.sum > 6, 1, Locn=util.current_fn(), err_param=str(self.sum)): self.errs.RecordErr()

    def calculate_product(self):
        self.product = self.x * self.y
        if self.errs.is_fail(self.product > 5, 1, Locn=util.current_fn(), err_param=str(self.product)): self.errs.RecordErr()

print('Sum, Product:', SumAndProduct(path_libs, 2, 3).procedure_to_do_all_steps)

ERROR: x * y should be less than or equal to 5. Product: 6
Sum, Product: (5, 6)


### Case 3: Performing Preflight Checks on Input Data

In [4]:
#Some sample input data to check
df = pd.DataFrame(data={'col_a':[1,2,3], 'col_b':[4,5,'a']})

#Instance a CheckDataFrame class with an ErrorHandle object as an attribute (self.errs)
ckdf = CheckDataFrame(df, ErrorHandle(path_libs, ErrMsgHeader='', IsPrint=True))

#Check that DataFrame col_b is all numeric
IsOk = ckdf.ColAllNumeric('col_b')
#Other programming steps or prefight checks...


ERROR: Column must contain only non-blank numeric values: col_b
